In [7]:
import pandas as pd

In [14]:
df = pd.read_csv("predictions/aluminum_openweight_results_nonull_debugged.csv")
df = df.drop(columns=["Unnamed: 0"])
df = df.dropna(subset=["model"])
df.head()

,doi,model,task,modality,AA,temper,YS [MPa],UTS [MPa],elong [%],Hardness,...,La,Li,Ni,P,Pb,Sc,Sn,Sr,V,Yb
0,10.1016/j.jmatprotec.2017.01.029,llama3.3-70,aluminum,xml,AA6061-T6,T6,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10.1016/j.jmatprotec.2017.01.029,llama3.3-70,aluminum,xml,AA6061,NaN,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10.1016/j.jmatprotec.2017.01.029,llama3.3-70,aluminum,xml,AA6061-T6,T6,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10.1016/j.jmatprotec.2017.01.029,llama3.3-70,aluminum,xml,NaN,NaN,449.6,379.8,0.012,0.9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10.1016/j.jmatprotec.2017.01.029,llama3.3-70,aluminum,xml,AA6061,T6,0.0,0.0,0.000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
df["modality"].fillna("missing").value_counts()

modality
images    8202
xml       6315
Name: count, dtype: int64

In [16]:
df["model"].value_counts()

model
gpt4o           4480
llama3.3-70     4291
molmo           3443
qwen2.5VL-72    2303
Name: count, dtype: int64

In [17]:
df.groupby(["model", "modality"]).count()

doi  task    AA  temper  YS [MPa]  UTS [MPa]  \
model        modality                                                  
gpt4o        images    2456  2456  1448    1554      2456       2456   
             xml       2024  2024  1362    1343      2024       2024   
llama3.3-70  xml       4291  4291  3470    2775      4291       4291   
molmo        images    3443  3443  3411    2823      3443       3443   
qwen2.5VL-72 images    2303  2303  1836    1723      2303       2303   

                       elong [%]  Hardness  Hardness UNIT  \
model        modality                                       
gpt4o        images         2456      2456            464   
             xml            2024      2024            345   
llama3.3-70  xml            4291      4291            402   
molmo        images         3443      3443           3321   
qwen2.5VL-72 images         2303      2303            879   

                       Has comp [True / False / nominal]  ...    La    Li  \
model        modality                                     ...               
gpt4o        images                                 2456  ...    55    67   
             xml                                    2024  ...    29    40   
llama3.3-70  xml                                    4291  ...  2825  2889   
molmo        images                                 3443  ...   398   317   
qwen2.5VL-72 images                                 2303  ...   798   832   

                         Ni     P    Pb    Sc    Sn    Sr     V    Yb  
model        modality                                                  
gpt4o        images     102    58    62   141    55    64    90    60  
             xml         58    29    35   141    29    45    39    30  
llama3.3-70  xml       2961  2876  2878  3195  3055  3107  3274  3442  
molmo        images     418    76   314   377   647   528   541   796  
qwen2.5VL-72 images     928   838   811   917   950   953  1017  1076  

[5 rows x 39 columns]

In [18]:
for group_name, group_idx  in df.groupby(["task", "model", "modality"]).groups.items():
    if pd.isna(group_name[0]):
        continue
    group_df = df.loc[group_idx]
    task, model, raw_modality = group_name
    if raw_modality == "xml":
        modality = "xml"
    else:
        modality = "pdf"
    group_df.rename(columns={"SDA_Type": "SDA Type"}).to_csv(f"predictions/{task}_{model}_{modality}.csv", index=False)
